In [34]:
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from src.average_precision import calculate_delta
import pandas as pd
from src.average_precision import convert_to_names
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost

In [2]:
X = pd.read_pickle("data/multiclass_training.pickle")

In [77]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X.drop(["fecha_dato","ncodpers", "target"], axis=1), X["target"], test_size=0.20, random_state=42)

In [78]:
xgbtrain = xgboost.DMatrix(X_train,y_train)
xgbtest = xgboost.DMatrix(X_test,y_test)
del X_train, X_test, y_train, y_test

In [79]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.05
param['max_depth'] = 8
param['silent'] = 0
param['num_class'] = 24
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 323
#param['eval_metric'] = "map@7"
num_rounds = 100

plst = list(param.items())
model = xgboost.train(plst, xgbtrain, num_rounds)

In [61]:
preds = model.predict(xgbtest)

In [74]:
model.save_model("models/xgboost.bin")